In [1]:
!pip install --upgrade git+https://github.com/pycaret/pycaret.git@2.2-refactor

     |████████████████████████████████| 210 kB 52.9 MB/s 
     |████████████████████████████████| 181 kB 47.7 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 75 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 56.3 MB/s 
  Created wheel for pycaret: filename=pycaret-2.2-py3-none-any.whl size=276552 sha256=877ac503ba42cc9e4e79b2060beb552180e44d0265b6a1d7b13286b9e1b1a764
  Stored in directory: /tmp/pip-ephem-wheel-cache-yjxiklum/wheels/6a/eb/78/df0d8f4b9c36d5a5b64a57a27750f9897dc40e5b1510e8f6a4
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=91585932b469c0756403bd248f43f6d705035a11f89d45218fe25315337312bb
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295206 sha256=3b8ce7c2160cf8e6d6c401067

In [ ]:
from pycaret.datasets import get_data
data = get_data('wine')

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red


In [ ]:
# randomly add one NaN to 75% of the rows. Rows cannot be repeated. shape[1]-1 to exclude target.
import numpy as np
random = np.random.default_rng(123)
shape = data.shape
size = shape[0] * shape[1]
idxs = set()
rows = set()
for _ in range(int(shape[0]*0.75)):
    x = (random.integers(0, shape[0]), random.integers(0, shape[1]-1))
    while x[0] in rows or x in idxs:
        x = (random.integers(0, shape[0]), random.integers(0, shape[1]-1))
    idxs.add(x)

In [ ]:
data_with_nans = data.copy()
for row, col in idxs:
    data_with_nans.iloc[row, col] = np.NaN

In [ ]:
data_without_nans = data_with_nans.dropna()
data_without_nans

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.400000,5.0,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.800000,6.0,red
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.99680,3.36,0.57,9.500000,7.0,red
13,7.8,0.61,0.29,1.6,0.114,9.0,29.0,0.99740,3.26,1.56,9.100000,5.0,red
14,8.9,0.62,0.18,3.8,0.176,52.0,145.0,0.99860,3.16,0.88,9.200000,5.0,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6468,6.6,0.24,0.33,10.1,0.032,8.0,81.0,0.99626,3.19,0.51,9.800000,6.0,white
6479,6.6,0.34,0.40,8.1,0.046,68.0,170.0,0.99494,3.15,0.50,9.533333,6.0,white
6481,5.5,0.32,0.13,1.3,0.037,45.0,156.0,0.99184,3.26,0.38,10.700000,5.0,white
6487,6.8,0.22,0.36,1.2,0.052,38.0,127.0,0.99330,3.04,0.54,9.200000,5.0,white


In [ ]:
data_with_nans.isna().sum()

fixed acidity           389
volatile acidity        421
citric acid             393
residual sugar          405
chlorides               419
free sulfur dioxide     433
total sulfur dioxide    446
density                 393
pH                      413
sulphates               397
alcohol                 388
quality                 375
type                      0
dtype: int64

In [ ]:
from time import time

In [ ]:
from pycaret.classification import *
time_start = time()
s = setup(data, target = 'type', train_size=0.99, session_id = 123, silent = True, ordinal_features={"quality": ["3","4","5","6","7","8","9"]},categorical_imputation="mode")
time_no_imputation = time() - time_start
train_no_nans = get_config('X_train')

,Description,Value
0,session_id,123
1,Target,type
2,Target Type,Binary
3,Label Encoded,"red: 0, white: 1"
4,Original Data,"(6497, 13)"
5,Missing Values,False
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,True
9,High Cardinality Features,False


In [ ]:
data_with_just_nans = train_no_nans[~train_no_nans.index.isin(data_without_nans.index)]
data_with_just_nans

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2990,6.2,0.16,0.47,1.4,0.029,23.0,81.0,0.99000,3.26,0.42,12.2,3.0
765,9.2,0.67,0.10,3.0,0.091,12.0,48.0,0.99888,3.31,0.54,9.5,3.0
2486,6.5,0.30,0.32,2.0,0.044,34.0,90.0,0.99185,3.37,0.68,11.0,4.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,2.0
2601,6.4,0.28,0.43,7.1,0.045,60.0,221.0,0.99520,3.09,0.45,9.4,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1593,6.8,0.62,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,3.0
5243,6.4,0.24,0.23,2.0,0.046,30.0,133.0,0.99080,3.12,0.54,11.4,4.0
6250,5.7,0.39,0.25,4.9,0.033,49.0,113.0,0.98966,3.26,0.58,13.1,4.0
5862,7.0,0.55,0.05,8.0,0.036,19.0,164.0,0.99269,3.26,0.46,12.2,3.0


In [ ]:
from pycaret.classification import *
time_start = time()
s = setup(data_with_nans, target = 'type', train_size=0.99, session_id = 123, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="simple", categorical_imputation="mode")
time_simple_imputation = time() - time_start
data_simple_imputation = get_config('X_train')
data_simple_imputation_just_nans = data_simple_imputation[~data_simple_imputation.index.isin(data_without_nans.index)]

,Description,Value
0,session_id,123
1,Target,type
2,Target Type,Binary
3,Label Encoded,"red: 0, white: 1"
4,Original Data,"(6497, 13)"
5,Missing Values,True
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,True
9,High Cardinality Features,False


In [ ]:
from pycaret.classification import *
time_start = time()
s = setup(data_with_nans, target = 'type', train_size=0.99, session_id = 123, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="iterative", categorical_imputation="mode", categorical_iterative_imputer="knn", numeric_iterative_imputer="knn")
time_knn_imputation = time() - time_start
data_iterative_imputation_knn = get_config('X_train')
data_iterative_imputation_knn_just_nans = data_iterative_imputation_knn[~data_iterative_imputation_knn.index.isin(data_without_nans.index)]

,Description,Value
0,session_id,123
1,Target,type
2,Target Type,Binary
3,Label Encoded,"red: 0, white: 1"
4,Original Data,"(6497, 13)"
5,Missing Values,True
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,True
9,High Cardinality Features,False


In [ ]:
from pycaret.classification import *
time_start = time()
s = setup(data_with_nans, target = 'type', train_size=0.99, session_id = 123, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="iterative", categorical_imputation="mode", categorical_iterative_imputer="rf", numeric_iterative_imputer="rf")
time_rf_imputation = time() - time_start
data_iterative_imputation = get_config('X_train')
data_iterative_imputation_just_nans = data_iterative_imputation[~data_iterative_imputation.index.isin(data_without_nans.index)]

,Description,Value
0,session_id,123
1,Target,type
2,Target Type,Binary
3,Label Encoded,"red: 0, white: 1"
4,Original Data,"(6497, 13)"
5,Missing Values,True
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,True
9,High Cardinality Features,False


In [ ]:
from pycaret.classification import *
time_start = time()
s = setup(data_with_nans, target = 'type', train_size=0.99, session_id = 123, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="iterative", categorical_imputation="mode", categorical_iterative_imputer="et", numeric_iterative_imputer="et")
time_et_imputation = time() - time_start
data_iterative_imputation_et = get_config('X_train')
data_iterative_imputation_et_just_nans = data_iterative_imputation_et[~data_iterative_imputation_et.index.isin(data_without_nans.index)]

,Description,Value
0,session_id,123
1,Target,type
2,Target Type,Binary
3,Label Encoded,"red: 0, white: 1"
4,Original Data,"(6497, 13)"
5,Missing Values,True
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,True
9,High Cardinality Features,False


In [ ]:
from pycaret.classification import *
time_start = time()
s = setup(data_with_nans, target = 'type', train_size=0.99, session_id = 123, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="iterative", categorical_imputation="mode", categorical_iterative_imputer="lightgbm", numeric_iterative_imputer="lightgbm")
time_lgbm_imputation = time() - time_start
data_iterative_imputation_lgbm = get_config('X_train')
data_iterative_imputation_lgbm_just_nans = data_iterative_imputation_lgbm[~data_iterative_imputation_lgbm.index.isin(data_without_nans.index)]

,Description,Value
0,session_id,123
1,Target,type
2,Target Type,Binary
3,Label Encoded,"red: 0, white: 1"
4,Original Data,"(6497, 13)"
5,Missing Values,True
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,True
9,High Cardinality Features,False


In [ ]:
from pycaret.classification import *
time_start = time()
s = setup(data_with_nans, target = 'type', train_size=0.99, session_id = 123, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="iterative", categorical_imputation="mode", categorical_iterative_imputer="catboost", numeric_iterative_imputer="catboost", iterative_imputation_iters=2)
time_cb_imputation = time() - time_start
data_iterative_imputation_cb = get_config('X_train')
data_iterative_imputation_cb_just_nans = data_iterative_imputation_cb[~data_iterative_imputation_cb.index.isin(data_without_nans.index)]

,Description,Value
0,session_id,123
1,Target,type
2,Target Type,Binary
3,Label Encoded,"red: 0, white: 1"
4,Original Data,"(6497, 13)"
5,Missing Values,True
6,Numeric Features,11
7,Categorical Features,1
8,Ordinal Features,True
9,High Cardinality Features,False


In [ ]:
from sklearn import metrics
from sklearn.utils.validation import check_consistent_length
from sklearn.metrics._regression import _check_reg_targets
def mean_absolute_percentage_error(
    y_true, y_pred, sample_weight=None, multioutput="uniform_average"
):
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput
    )
    check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true)
    output_errors = np.average(mape, weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == "raw_values":
            return output_errors
        elif multioutput == "uniform_average":
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)
def highlight_min(s):
    to_highlight = s == s.min()
    return ["background-color: yellow" if v else "" for v in to_highlight]

idx = ["simple", "iterative_rf","iterative_et","iterative_knn", "iterative_lightgbm", "iterative_catboost"]

In [ ]:
# calculate MSE, MAPE for imputation
from sklearn.metrics import mean_squared_error
from collections import defaultdict
mse_dict = defaultdict(list)
mape_dict = defaultdict(list)
for col in data_with_just_nans.columns:
    mse_dict[col].append(mean_squared_error(data_with_just_nans[col], data_simple_imputation_just_nans[col]))
    mse_dict[col].append(mean_squared_error(data_with_just_nans[col], data_iterative_imputation_just_nans[col]))
    mse_dict[col].append(mean_squared_error(data_with_just_nans[col], data_iterative_imputation_et_just_nans[col]))
    mse_dict[col].append(mean_squared_error(data_with_just_nans[col], data_iterative_imputation_knn_just_nans[col]))
    mse_dict[col].append(mean_squared_error(data_with_just_nans[col], data_iterative_imputation_lgbm_just_nans[col]))
    mse_dict[col].append(mean_squared_error(data_with_just_nans[col], data_iterative_imputation_cb_just_nans[col]))

    mape_dict[col].append(mean_absolute_percentage_error(data_with_just_nans[col], data_simple_imputation_just_nans[col]))
    mape_dict[col].append(mean_absolute_percentage_error(data_with_just_nans[col], data_iterative_imputation_just_nans[col]))
    mape_dict[col].append(mean_absolute_percentage_error(data_with_just_nans[col], data_iterative_imputation_et_just_nans[col]))
    mape_dict[col].append(mean_absolute_percentage_error(data_with_just_nans[col], data_iterative_imputation_knn_just_nans[col]))
    mape_dict[col].append(mean_absolute_percentage_error(data_with_just_nans[col], data_iterative_imputation_lgbm_just_nans[col]))
    mape_dict[col].append(mean_absolute_percentage_error(data_with_just_nans[col], data_iterative_imputation_cb_just_nans[col]))

In [ ]:
results_mse = pd.DataFrame(mse_dict, index=idx)
results_mse["MEAN"] = results_mse.mean(axis=1)
results_mse.round(6).style.apply(highlight_min)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,MEAN
simple,0.196991,0.003472,0.002255,3.532953,0.000130,42.225189,377.240387,0.000001,0.002897,0.003266,0.154479,0.087518,35.287461
iterative_rf,0.034490,0.001059,0.000970,0.868314,0.000037,12.309838,58.307268,0.000000,0.000840,0.000940,0.020663,0.044763,5.965765
iterative_et,0.026121,0.000892,0.000846,1.038144,0.000030,11.143758,51.344097,0.000000,0.000727,0.000740,0.019188,0.044189,5.301561
iterative_knn,0.136063,0.002276,0.002110,2.946783,0.000098,19.825018,130.955124,0.000000,0.002259,0.002441,0.129615,0.108752,12.842545
iterative_lightgbm,0.031393,0.001063,0.001112,0.995042,0.000038,14.148662,68.244779,0.000000,0.000864,0.001131,0.018919,0.052511,6.957960
iterative_catboost,0.028789,0.000998,0.001067,1.109552,0.000037,12.006286,61.428720,0.000000,0.000793,0.000988,0.017460,0.050502,6.220433


In [ ]:
results = pd.DataFrame(mape_dict, index=idx)
results["MEAN"] = results.mean(axis=1)
results.round(6).style.apply(highlight_min)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,MEAN
simple,0.103606,0.015374,0.011787,0.462540,0.002399,1.790353,5.616408,0.000289,0.014386,0.013291,0.107455,0.068006,0.683824
iterative_rf,0.043379,0.006640,0.006954,0.115652,0.001001,0.772740,1.964049,0.000058,0.007078,0.007109,0.032357,0.035007,0.249335
iterative_et,0.033480,0.005648,0.005742,0.103038,0.000819,0.650568,1.656184,0.000038,0.005713,0.005565,0.027544,0.035007,0.210779
iterative_knn,0.091495,0.011678,0.011361,0.349627,0.001736,1.119570,3.069412,0.000130,0.012587,0.011043,0.088477,0.071449,0.403214
iterative_lightgbm,0.044989,0.007589,0.007861,0.117039,0.001187,0.902421,2.260691,0.000049,0.007745,0.008205,0.032567,0.041607,0.285996
iterative_catboost,0.041718,0.007357,0.007682,0.114456,0.001191,0.850199,2.142914,0.000049,0.007232,0.007737,0.031743,0.041894,0.271181


In [ ]:
times = pd.DataFrame([time_simple_imputation, time_rf_imputation, time_et_imputation, time_knn_imputation, time_lgbm_imputation, time_cb_imputation], index=idx, columns=["time (ms)"])
times["difference with simple"] = times-times.loc["simple"]

,time (ms),difference with simple
simple,0.779323,0.000000
iterative_rf,124.573176,123.793853
iterative_et,77.423099,76.643775
iterative_knn,9.142633,8.363310
iterative_lightgbm,27.354731,26.575407
iterative_catboost,118.892360,118.113037


In [ ]:
final_results = pd.DataFrame([results["MEAN"], times["difference with simple"]]).T
final_results.columns = ["Mean MAPE", "Time difference with simple"]
final_results["MAPE loss over simple"] = final_results["Mean MAPE"]-final_results["Mean MAPE"].loc["simple"]
final_results["MAPE loss per s"] = final_results["MAPE loss over simple"]/final_results["Time difference with simple"]
final_results.sort_values(by="Mean MAPE",inplace=True)
final_results

,Mean MAPE,Time difference with simple,MAPE loss over simple,MAPE loss per s
iterative_et,0.210779,76.643775,-0.473046,-0.006172
iterative_rf,0.249335,123.793853,-0.434489,-0.003510
iterative_catboost,0.271181,118.113037,-0.412643,-0.003494
iterative_lightgbm,0.285996,26.575407,-0.397829,-0.014970
iterative_knn,0.403214,8.363310,-0.280611,-0.033553
simple,0.683824,0.000000,0.000000,NaN


In [ ]:
import plotly.express as px
fig = px.bar(final_results["Mean MAPE"], color=final_results.index, title="Mean MAPE of imputation (smaller is better)")
fig.show()

In [ ]:
import plotly.express as px
fig = px.bar(final_results.drop("simple")["MAPE loss per s"], color=final_results.drop("simple").index, title="Mean MAPE of imputation loss per s (smaller is better)")
fig.show()

In [ ]:
fig = px.scatter(final_results.drop("simple"), x="Time difference with simple", y="MAPE loss over simple", color=final_results.drop("simple").index, title="Iterative Imputers")
fig.update_traces(mode='markers', marker_size=20)
fig.show()

In [ ]:
from pycaret.classification import *
simple_res=[]
test_df = data.sample(frac=0.3, random_state=42)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df_nans, test_data=test_df, target = 'type', train_size=0.99, session_id = 42, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="simple", categorical_imputation="mode", categorical_iterative_imputer="et", numeric_iterative_imputer="et")
create_model("dt", cross_validation=False)
simple_res.append(pull().iloc[0])
test_df = data.sample(frac=0.3, random_state=43)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df_nans, test_data=test_df, target = 'type', train_size=0.99, session_id = 43, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="simple", categorical_imputation="mode", categorical_iterative_imputer="et", numeric_iterative_imputer="et")
create_model("dt", cross_validation=False)
simple_res.append(pull().iloc[0])
test_df = data.sample(frac=0.3, random_state=44)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df_nans, test_data=test_df, target = 'type', train_size=0.99, session_id = 44, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="simple", categorical_imputation="mode", categorical_iterative_imputer="et", numeric_iterative_imputer="et")
create_model("dt", cross_validation=False)
simple_res.append(pull().iloc[0])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9872,0.9858,0.9858,0.9873,0.9872,0.9653,0.9654


In [ ]:
from pycaret.classification import *
iterative_res=[]
test_df = data.sample(frac=0.3, random_state=42)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df_nans, test_data=test_df, target = 'type', train_size=0.99, session_id = 42, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="iterative", categorical_imputation="mode", iterative_imputation_iters=10)
create_model("dt", cross_validation=False)
iterative_res.append(pull().iloc[0])
test_df = data.sample(frac=0.3, random_state=43)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df_nans, test_data=test_df, target = 'type', train_size=0.99, session_id = 43, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="iterative", categorical_imputation="mode",  iterative_imputation_iters=10)
create_model("dt", cross_validation=False)
iterative_res.append(pull().iloc[0])
test_df = data.sample(frac=0.3, random_state=44)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df_nans, test_data=test_df, target = 'type', train_size=0.99, session_id = 44, silent = True, ordinal_features={"quality": ["3.0","4.0","5.0","6.0","7.0","8.0","9.0"]}, imputation_type="iterative", categorical_imputation="mode",  iterative_imputation_iters=10)
create_model("dt", cross_validation=False)
iterative_res.append(pull().iloc[0])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9877,0.9883,0.9883,0.9879,0.9877,0.9668,0.967


In [ ]:
from pycaret.classification import *
base_res = []
test_df = data.sample(frac=0.3, random_state=42)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df, test_data=test_df, target = 'type', train_size=0.99, session_id = 42, silent = True, ordinal_features={"quality": ["3","4","5","6","7","8","9"]})
create_model("dt", cross_validation=False)
base_res.append(pull().iloc[0])
test_df = data.sample(frac=0.3, random_state=43)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df, test_data=test_df, target = 'type', train_size=0.99, session_id = 43, silent = True, ordinal_features={"quality": ["3","4","5","6","7","8","9"]})
create_model("dt", cross_validation=False)
base_res.append(pull().iloc[0])
test_df = data.sample(frac=0.3, random_state=44)
train_df = data[~data.index.isin(test_df.index)]
train_df_nans = data_with_nans[~data_with_nans.index.isin(test_df.index)]
s = setup(train_df, test_data=test_df, target = 'type', train_size=0.99, session_id = 44, silent = True, ordinal_features={"quality": ["3","4","5","6","7","8","9"]})
create_model("dt", cross_validation=False)
base_res.append(pull().iloc[0])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9851,0.9829,0.9823,0.9852,0.9852,0.9597,0.9598


In [ ]:
def get_means(dfs):
    df = pd.DataFrame(pd.concat(dfs))
    return df.groupby(df.index).mean()
results = pd.concat([get_means(x) for x in (simple_res, iterative_res, base_res)], axis=1)
results.columns=["Simple most frequent", "Iterative lightgbm", "Full data"]
results=results.T
def highlight_max(s):
    to_highlight = s == s.max()
    return ["background-color: yellow" if v else "" for v in to_highlight]
results.style.apply(highlight_max)

,AUC,Accuracy,F1,Kappa,MCC,Prec.,Recall
Simple most frequent,0.983300,0.986167,0.986167,0.963167,0.963233,0.986233,0.983300
Iterative lightgbm,0.985367,0.987500,0.987500,0.966833,0.966900,0.987600,0.985367
Full data,0.983233,0.986333,0.986367,0.963533,0.963567,0.986333,0.982900
